In [51]:
import pandas as pd
import numpy as np
import verispy, os
from verispy import VERIS

[Source for VERISPY + Useful README](https://github.com/RiskLens/verispy)

In [69]:


DATA_DIR ="./data/validated/"
PKL_PATH = "./data/vdf.pkl"

v = VERIS(json_dir=DATA_DIR, verbose=True)

# json_to_df is an expensive function, so we save the converted dataframe for use between sessions. 
if not os.path.exists(PKL_PATH):
    print("we're making pickles")
    vdf = v.json_to_df()
    vdf.to_pickle(PKL_PATH)
vdf = pd.read_pickle(PKL_PATH)
# v.raw_df=vdf
print(vdf.shape)

v.enum_summary(vdf, "action")


Found 8323 json files.
(8323, 2347)


,enum,x,n,freq
0,Error,2418,8105.0,0.29833
1,Hacking,2213,8105.0,0.27304
2,Misuse,1697,8105.0,0.20938
3,Physical,1576,8105.0,0.19445
4,Malware,697,8105.0,0.08600
5,Social,568,8105.0,0.07008
6,Environmental,8,8105.0,0.00099
7,Unknown,218,NaN,NaN


##### v is a VERIS object. vdf contains the actual data. 

[Good StackOverflow Answer on working with Dataframes vvv](https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values)

In [67]:
df_impact = vdf.loc[:, vdf.columns.str.contains('rating')]
print(v.enum_summary(vdf, 'impact.overall_rating'))
print(v.enum_summary(vdf, "impact.loss.rating"))


# v.enum_summary(vdf, "impact.rating"

            enum     x      n     freq
0  Insignificant    52  116.0  0.44828
1    Distracting    25  116.0  0.21552
2       Damaging    20  116.0  0.17241
3        Painful    18  116.0  0.15517
4   Catastrophic     1  116.0  0.00862
5        Unknown  5767    NaN      NaN
       enum  x    n  freq
0     Major  0  0.0   NaN
1     Minor  0  0.0   NaN
2  Moderate  0  0.0   NaN
3      None  0  0.0   NaN
4   Unknown  0  NaN   NaN


In [58]:
df_action = vdf.loc[:, vdf.columns.str.startswith('action')]


In [99]:
df_final = df_action.iloc[:, :7]

In [102]:
df_bugs = vdf[vdf['action.Hacking'] | vdf['action.Malware']]
# print(df_bugs.shape)



In [61]:
indices = list(df_bugs.index)
bugs = [False] * df_final.shape[0]
for i in indices:
    bugs[i] = True

In [62]:
df_final['action.Bugs'] = bugs

In [63]:
df_final

,action.Environmental,action.Error,action.Hacking,action.Malware,action.Misuse,action.Physical,action.Social,action.Bugs
0,False,False,False,False,False,True,False,False
1,False,True,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False
3,False,False,False,False,True,False,False,False
4,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...
8318,False,False,True,False,False,False,False,True
8319,False,False,False,False,True,False,True,False
8320,False,False,False,False,False,True,False,False
8321,False,True,False,False,False,False,False,False


In [105]:

N = 1000
sample = np.random.choice(8323, N)
df_short = df_final.iloc[sample,:].reset_index().iloc[:,1:]


# Maybe we should id the column here in the future so we can refer to the original table values

In [106]:
df_short

,action.Environmental,action.Error,action.Hacking,action.Malware,action.Misuse,action.Physical,action.Social
0,False,True,False,False,False,False,False
1,False,False,True,False,False,False,False
2,False,False,True,False,False,False,False
3,False,False,False,False,True,False,False
4,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...
995,False,False,True,True,True,True,False
996,False,False,False,False,False,True,False
997,False,True,False,False,False,False,False
998,False,False,True,False,False,False,False
